## Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

### Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). 

Получившийся список должен содержать в себе минимум: Наименование вакансии. Предлагаемую зарплату (отдельно минимальную и максимальную). Ссылку на саму вакансию.

Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
import requests
import json
import lxml
from bs4 import BeautifulSoup as bs
import pandas as pd
from pymongo import MongoClient

функции парсинга зарплаты и вакансии вцелом

In [2]:
def parse_zp(string, dict_zp):
    if len(string) == 13:
        dict_zp['min zp'] = int(string[2].replace('\xa0',''))
        dict_zp['max zp'] = int(string[6].replace('\xa0',''))
        dict_zp['curr'] = string[10]
        dict_zp['prim'] = string[12]
    elif str(string[0]) == 'от ':
            dict_zp['min zp'] = int(string[2].replace('\xa0',''))
            dict_zp['curr'] = string[6]
            dict_zp['prim'] = string[8]
    elif str(string[0]) == 'до ':
            dict_zp['max zp'] = int(string[2].replace('\xa0',''))
            dict_zp['curr'] = string[6]
            dict_zp['prim'] = string[8]

    return dict_zp
    

def parse_vac(url):
    dict_vac = {'name': None, 'min zp': None, 'max zp': None, 'curr': None, 'prim': None, 'url': url}
    headers = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
    req = requests.get(url, headers=headers).text
    soup = bs(req)
    dict_vac = parse_zp(soup.find(attrs = {'data-qa': 'bloko-header-2'}).contents,  dict_vac)
    dict_vac['name'] = soup.find(attrs = {'data-qa': 'vacancy-title'}).contents[0]
    
    return dict_vac
    

In [3]:
vac = 'Специалист Data Science' # вакансия
n = 5 # количество вакансий

Получение списка страниц заданной вакансии

In [4]:
req = requests.get(f'https://api.hh.ru/vacancies?text={vac}&only_with_salary=true&currency=RUR&User-Agent=HH-User-Agent')

In [5]:
data = json.loads(req.text) #переводим в список

In [6]:
list_of_vac=[]        #очищаем от прошлых результатов
for i in range(n):    #формируем список из n ссылок
     list_of_vac.append(parse_vac(data['items'][i]['alternate_url']))
#print(list_of_vac)
df = pd.DataFrame(list_of_vac)
df

,name,min zp,max zp,curr,prim,url
0,Специалист Data Science,100000.0,200000.0,руб.,на руки,https://hh.ru/vacancy/46857375
1,Data Scientist,NaN,300000.0,руб.,на руки,https://hh.ru/vacancy/45191074
2,Data Scientist,100000.0,250000.0,руб.,на руки,https://hh.ru/vacancy/44350010
3,Data Scientist,100000.0,NaN,руб.,на руки,https://hh.ru/vacancy/48118086
4,Data Scientist (Middle),140000.0,160000.0,руб.,на руки,https://hh.ru/vacancy/47765295


## Урок 3. Системы управления базами данных MongoDB и SQLite в Python

### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.

In [ ]:
client = MongoClient('localhost', 27017)
db = client['vacancies']
collection = db.DataScientists

### 3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [ ]:
for el in list_of_vac:
    if len(list(db.DataScientists.find({ 'url': el['url'] }))) == 0:
        db.DataScientists.insert_one(el) 

### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

In [ ]:
user_zp = 200000 #Пороговое значение зарплаты
answ_zp=db.DataScientists.find( { '$or': [ { 'min zp': { '$gt': user_zp } }, { 'max zp': { '$gt': user_zp } } ] } )
df1 = pd.DataFrame(list(answ_zp))
df1

In [ ]:
#db.DataScientists.drop()